In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('spamham').getOrCreate()

In [2]:
df = spark.read.csv('FileStore/tables/SMSSpamCollection', inferSchema=True, sep='\t')

In [3]:
df = df.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [4]:
df.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

In [5]:
from pyspark.sql.functions import length

In [6]:
df = df.withColumn('length', length(df['text']))

In [7]:
df.show(5)

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
+-----+--------------------+------+
only showing top 5 rows

In [8]:
df.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [9]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [10]:
tokenizer= Tokenizer(inputCol='text', outputCol='tokenText')
stop_rem = StopWordsRemover(inputCol='tokenText', outputCol='stop_token')
cv = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
cleanup = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [13]:
from pyspark.ml.classification import NaiveBayes

In [14]:
nb = NaiveBayes()

In [15]:
from pyspark.ml import Pipeline

In [16]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num, tokenizer, stop_rem, cv, idf, cleanup])

In [17]:
final_df = data_prep_pipe.fit(df).transform(df)

In [18]:
final_df = final_df.select(['label', 'features'])

In [19]:
final_df.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows

In [20]:
train_df, test_df = final_df.randomSplit([0.7,0.3])

In [21]:
spam_det = nb.fit(train_df)

In [22]:
results = spam_det.transform(test_df)

In [23]:
results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,5,20,...|[-809.68633839681...|[1.0,2.0615806509...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-883.63226463192...|[1.0,6.3442576885...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-564.67328899948...|[1.0,3.0367252045...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-564.67328899948...|[1.0,3.0367252045...| 0.0|
 0.0|(13424,[0,1,11,32...|[-878.45263860034...|[1.0,2.3315359660...| 0.0|
 0.0|(13424,[0,1,14,31...|[-216.81839513690...|[1.0,9.3529012879...| 0.0|
 0.0|(13424,[0,1,27,35...|[-1491.3602129118...|[0.99999983589715...| 0.0|
 0.0|(13424,[0,2,3,5,3...|[-498.42059009121...|[1.0,3.3950066536...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3288.0481689782...|[1.0,1.0730190448...| 0.0|
 0.0|(13424,[0,2,4,7,2...|[-513.30521622020...|[1.0,4.4732599227...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-548.29659235714...|[1.0,2.1403705200...| 0.0|
 0.0|(13424,[0,2,4,25,...|[-426.93952857651...|[1.0,2.3700688606...| 0.0|
 0.0|(13424,[0,2,4,44,...|[-1892.7576971950...|[1.0,9.0543505996...| 0.0|
 0.0|(13424,[0,2,5,8,4...|[-830.61455425658...|[1.0,3.9616869317...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[-685.45244325249...|[1.0,1.0080644989...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-837.06790427352...|[1.0,5.9943053866...| 0.0|
 0.0|(13424,[0,2,7,43,...|[-586.79673032910...|[1.0,1.2857151058...| 0.0|
 0.0|(13424,[0,2,7,95,...|[-632.37429881336...|[1.0,1.8584784532...| 0.0|
 0.0|(13424,[0,2,7,114...|[-448.14035543201...|[1.0,5.8831275214...| 0.0|
 0.0|(13424,[0,2,8,25,...|[-308.86533897917...|[1.0,1.0471819514...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
acc_eval = MulticlassClassificationEvaluator()

In [26]:
acc = acc_eval.evaluate(results)

In [27]:
acc

Out[ 27 ]: 0.9187878277876754